# Analyse der Plenarprotokolle - TextBlob
## Dataset preparation - Kommentar

In [29]:
import pickle, gensim, numpy as np
import pandas as pd 

PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_plenar/'
df = pickle.load(open(PICKLE_FOLDER_PATH + 'df_comment_date_all.pickle', "rb" ))


##### Bearbeiten bzw vereinheitlichen der Fraktions Spalte

In [41]:
import re
df = df[df['Fraktion'] != 'Bremen']
df['Fraktion'] = df['Fraktion'].replace(to_replace=r'(BÜNDNIS(SES)*\s90\/DIE\sGRÜNEN)[$:,]*(?i)', value='BÜNDNIS_90/DIE_GRÜNEN', regex=True)
df['Fraktion'] = df['Fraktion'].replace(to_replace=r'((DIE )*LINKE)[N$:,]*', value='DIE_LINKE', regex=True)
df['Fraktion'] = df['Fraktion'].replace(to_replace=r'(Fraktionslos(?i))', value='Fraktionslos', regex=True)
df['Fraktion'] = df['Fraktion'].replace(to_replace=r'(SPD.*)', value='SPD', regex=True)

/usr/local/anaconda3/bin/ipython:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # -*- coding: utf-8 -*-
/usr/local/anaconda3/bin/ipython:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import re
/usr/local/anaconda3/bin/ipython:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

##### Erstelle Dictionary mit Kommentaren und Text Cleaning

In [ ]:
parteien = ['SPD', 'FDP', 'AfD', 'CDU/CSU', 'BÜNDNIS_90/DIE_GRÜNEN', 'DIE_LINKE']

dic_kommentare = {}
for partei in parteien:
    df_comment = df[['Rede_ID', partei+'_Text1', partei+'_Text2']]    
    
    dic_kommentare[partei+'_Text1'] = df_comment.loc[df_comment[partei+'_Text1']!=''][['Rede_ID', partei+'_Text1']]
    dic_kommentare[partei+'_Text1'].index = dic_kommentare[partei+'_Text1']['Rede_ID']
    dic_kommentare[partei+'_Text1'][partei+'_Text1'] = dic_kommentare[partei+'_Text1'][partei+'_Text1'].map(lambda x: re.sub('§[\n\t,\.!?#@\\n"“„\:;&\(\)]', '', x))
    dic_kommentare[partei+'_Text1'][partei+'_Text1'] = dic_kommentare[partei+'_Text1'][partei+'_Text1'].map(lambda x: x.lower())
    
    dic_kommentare[partei+'_Text2'] = df_comment.loc[df_comment[partei+'_Text2']!=''][['Rede_ID', partei+'_Text2']]
    dic_kommentare[partei+'_Text2'].index = dic_kommentare[partei+'_Text2']['Rede_ID']
    dic_kommentare[partei+'_Text2'][partei+'_Text2'] = dic_kommentare[partei+'_Text2'][partei+'_Text2'].map(lambda x: re.sub('§[\n\t,\.!?#@\\n"“„\:;&\(\)]', '', x))
    dic_kommentare[partei+'_Text2'][partei+'_Text2'] = dic_kommentare[partei+'_Text2'][partei+'_Text2'].map(lambda x: x.lower())

In [ ]:
parteien = ['SPD', 'FDP', 'AfD', 'CDU/CSU', 'BÜNDNIS_90/DIE_GRÜNEN', 'DIE_LINKE']

dic_kommentare = {}
for partei in parteien:
    df_comment = df[['Rede_ID', partei+'_Text1', partei+'_Text2', partei+'_Akteur1', partei+'_Akteur2', 'Beschreibung2']]    
    
    dic_kommentare[partei+'_Akteur1'] = df_comment.loc[df_comment[partei+'_Text1']!=''][['Rede_ID', partei+'_Akteur1', partei+'_Text1', 'Beschreibung2']]
    dic_kommentare[partei+'_Akteur1'][partei+'_Text1'] = dic_kommentare[partei+'_Akteur1'][partei+'_Text1'].map(lambda x: re.sub('§[\n\t,\.!?#@\\n"“„\:;&\(\)]', '', x))
    dic_kommentare[partei+'_Akteur1'][partei+'_Text1'] = dic_kommentare[partei+'_Akteur1'][partei+'_Text1'].map(lambda x: x.lower())
    
    dic_kommentare[partei+'_Akteur2'] = df_comment.loc[df_comment[partei+'_Text2']!=''][['Rede_ID', partei+'_Akteur2', partei+'_Text2', 'Beschreibung2']]
    dic_kommentare[partei+'_Akteur2'][partei+'_Text2'] = dic_kommentare[partei+'_Akteur2'][partei+'_Text2'].map(lambda x: re.sub('§[\n\t,\.!?#@\\n"“„\:;&\(\)]', '', x))
    dic_kommentare[partei+'_Akteur2'][partei+'_Text2'] = dic_kommentare[partei+'_Akteur2'][partei+'_Text2'].map(lambda x: x.lower())

In [ ]:
for partei in parteien:
    for t in ['Akteur1', 'Akteur2']:
        # Überarbeite dfs in dic (Index, Columnnames, Parteienspalte)
        dic_kommentare[partei+'_'+t].columns = [['Rede_ID', 'Akteur', 'Text', 'Beschreibung2']]
        dic_kommentare[partei+'_'+t]['Partei'] = partei
        
# Füge dic Einträge zusammen in einen DataFrame
df_kommentar = pd.concat(dic_kommentare, ignore_index=True, axis = 0)
# Flatten den Multiindex des df
df_kommentar.columns = df_kommentar.columns.get_level_values(0)

df_kommentar

In [13]:
import pickle, gensim, numpy as np
import pandas as pd 

PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_TUWien_plenar/'
#pickle.dump(df_kommentar, open(PICKLE_FOLDER_PATH + 'Plenum_Kom_Sent_Wien_TB_Akteur', "wb" ))

df_kommentar = pickle.load(open(PICKLE_FOLDER_PATH + 'Plenum_Kom_Sent_Wien_TB_Akteur', "rb" ))

## Dataset preparation - Reden

In [48]:
#sample_df = time_sample_df
sample_df = df[df['Rede_Text'].isna() != True]

# Remove punctuation
sample_df['full_text_processed'] = sample_df['Rede_Text'].map(lambda x: re.sub('§[\n\t,\.!?#@\\n"“„\:;&\(\)]', '', x))

# # Convert the titles to lowercase
sample_df['full_text_processed'] = sample_df['full_text_processed'].map(lambda x: x.lower())

sample_df = sample_df[sample_df['full_text_processed'] != '']
sample_df = sample_df[sample_df['full_text_processed'] != ' ']
sample_df = sample_df[sample_df['full_text_processed'] != '   ']

df_Reden = sample_df[['Rede_ID', 'full_text_processed']]
df_Reden.columns = ['Rede_ID', 'Text']
df_Reden = df_Reden.groupby('Rede_ID')['Text'].apply(' '.join).reset_index()

/usr/local/anaconda3/bin/ipython:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/anaconda3/bin/ipython:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Sentiment Analyse
## TextBlob auf Kommentare angewandt

In [ ]:
# https://medium.com/@himanshu_23732/sentiment-analysis-with-textblob-6bc2eb9ec4ab
from textblob_de import TextBlobDE as TextBlob

def sentiment(text):
    try:
        return TextBlob(text).sentiment
    except:
        return None
    
for partei in parteien:
    dic_kommentare[partei+'_Text1']['Polarity'] = dic_kommentare[partei+'_Text1'][partei+'_Text1'].apply(sentiment).apply(lambda x: x[0])
    dic_kommentare[partei+'_Text1']['Subjectivity'] = dic_kommentare[partei+'_Text1'][partei+'_Text1'].apply(sentiment).apply(lambda x: x[1])
    dic_kommentare[partei+'_Text2']['Polarity'] = dic_kommentare[partei+'_Text2'][partei+'_Text2'].apply(sentiment).apply(lambda x: x[0])
    dic_kommentare[partei+'_Text2']['Subjectivity'] = dic_kommentare[partei+'_Text2'][partei+'_Text2'].apply(sentiment).apply(lambda x: x[1])


In [17]:
import pickle, gensim, numpy as np
import pandas as pd 

PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_TUWien_plenar/'
# pickle.dump(dic_kommentare, open(PICKLE_FOLDER_PATH + 'Plenum_Kom_Sent_Wien_TB_Akteur', "wb" ))
dic_kommentare = pickle.load(open(PICKLE_FOLDER_PATH + 'Plenum_Kom_Sent_Wien_TB', "rb" ))

## TextBlob auf Reden angewandt

In [49]:
# https://medium.com/@himanshu_23732/sentiment-analysis-with-textblob-6bc2eb9ec4ab
from textblob_de import TextBlobDE as TextBlob

def sentiment(text):
    try:
        return TextBlob(text).sentiment
    except:
        return None

sentiment(df_Reden['Text'][0])
df_Reden['Polarity'] = df_Reden['Text'].apply(sentiment).apply(lambda x: x[0])

In [51]:
pd.set_option('display.max_colwidth', None)
df_Reden.columns

Index(['Rede_ID', 'Text', 'Polarity'], dtype='object')

## Übersetzen von 'Polarity' in Categorie (Pos, Neut, Neg)
## Erstelle Spalte 'Category' für Kommentare

In [18]:
# compute sentiment scores (polarity) and labels
parteien = ['SPD', 'FDP', 'AfD', 'CDU/CSU', 'BÜNDNIS_90/DIE_GRÜNEN', 'DIE_LINKE']
for partei in parteien:
    for t in ['Text1', 'Text2']:
        sentiment_scores = dic_kommentare[partei+'_'+t]['Polarity']
        # Überarbeite dfs in dic (Index, Columnnames, Parteienspalte)
        dic_kommentare[partei+'_'+t].index = np.arange(0, len(dic_kommentare[partei+'_'+t]))
        dic_kommentare[partei+'_'+t].columns = [['Rede_ID', 'Text', 'Polarity', 'Subjectivity']]
        dic_kommentare[partei+'_'+t]['Partei'] = partei

        dic_kommentare[partei+'_'+t]['category'] = ['positive' if score > 0 
                                 else 'negative' if score < 0 
                                     else 'neutral' 
                                         for score in sentiment_scores]

In [20]:
# Füge dic Einträge zusammen in einen DataFrame
df_kommentar = pd.concat(dic_kommentare, ignore_index=True, axis = 0)
# Flatten den Multiindex des df
df_kommentar.columns = df_kommentar.columns.get_level_values(0)

In [24]:
import re ###########################

# Initiiere die neuen Spalten
df_kommentar['Datum'] = 'NaN'
df_kommentar['Fraktion'] = 'NaN'
df_kommentar['Nachname'] = 'NaN'
df_kommentar['Beschreibung1'] = 'NaN'
df_kommentar['Beschreibung2'] = 'NaN'

for ID, n in zip(df_kommentar['Rede_ID'], range(0, len(df_kommentar['Rede_ID']))):
    i = df[df['Rede_ID'] == ID].index[1] # Ermittle die erste Zeile im Ursprungsdatensatz mit gesuchter ID
    
    df_kommentar['Datum'][n] = df[df['Rede_ID'] == ID]['Datum'][i]
    df_kommentar['Fraktion'][n] = df[df['Rede_ID'] == ID]['Fraktion'][i]
    df_kommentar['Nachname'][n] = df[df['Rede_ID'] == ID]['Nachname'][i]
    df_kommentar['Beschreibung1'][n] = df[df['Rede_ID'] == ID]['Beschreibung1'][i]
    df_kommentar['Beschreibung2'][n] = df[df['Rede_ID'] == ID]['Beschreibung2'][i]

# Erstelle Spalte für Monat und Woche aus Datum
df_kommentar.loc[:,('Monat')] = df_kommentar.Datum.dt.strftime('%y-%m')
df_kommentar.loc[:,('Woche')] = df_kommentar.Datum.dt.strftime('%y-w%U')

# Vereinhetliche Einträge in Spalte 'Fraktion' (= Partei des Redners)
df_kommentar['Fraktion'] = df_kommentar['Fraktion'].replace(to_replace=r'(BÜNDNIS.*)(?i)', value='BÜNDNIS_90/DIE_GRÜNEN', regex=True)
df_kommentar['Fraktion'] = df_kommentar['Fraktion'].replace(to_replace=r'(Fraktionslos.*)(?i)', value='Fraktionslos', regex=True)

/usr/local/anaconda3/bin/ipython:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/anaconda3/bin/ipython:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/anaconda3/bin/ipython:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/anaconda3/bin/ipython:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [ ]:
import pickle, gensim, numpy as np
import pandas as pd 

PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_TUWien_plenar/'
#pickle.dump(df_kommentar, open(PICKLE_FOLDER_PATH + 'Plenum_Kom_TB', "wb" ))
df_kommentar = pickle.load(open(PICKLE_FOLDER_PATH + 'Plenum_Kom_TB', "rb" ))
df_kommentar # Fertiger DataFrame

## Erstelle Spalte 'Category' für Reden

In [52]:
# compute sentiment scores (polarity) and labels
# save_df1 = df_Reden
sentiment_scores = df_Reden['Polarity']

df_Reden['category'] = ['positive' if score > 0.05
                                 else 'negative' if score < -0.05
                                     else 'neutral' 
                                         for score in sentiment_scores]

In [56]:
import pickle, gensim, numpy as np
import pandas as pd 

PICKLE_FOLDER_PATH = '/home/lisa/Darmstadt/Master Arbeit/06_Analyse/Plenum/pickle_TUWien_plenar/'
#pickle.dump(df_Reden, open(PICKLE_FOLDER_PATH + 'Plenum_Rede_Sent_TB', "wb" ))
df_Reden = pickle.load(open(PICKLE_FOLDER_PATH + 'Plenum_Rede_Sent_TB', "rb" ))
df_Reden # Fertiger DataFrame

Rede_ID  \
0      ID1910000100   
1      ID1910000200   
2      ID1910000300   
3      ID1910000400   
4      ID1910000500   
...             ...   
17080   ID199906800   
17081   ID199906900   
17082   ID199907000   
17083   ID199907100   
17084   ID199907200   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Darstellung von Ergebnissen

In [ ]:
import pandas as pd
mdp_partei_TB = df_kommentar
#mdp_partei = mdp_partei.loc[(mdp_partei['predict'] == 'OFFENSE') & (mdp_partei['date'] >= datetime.date(2019,10,25))]
partei_piv_TB = pd.pivot_table(mdp_partei_TB, columns = ['category'], index = ['Fraktion'], aggfunc = np.count_nonzero)
partei_piv_TB['Text']

In [ ]:
import pandas as pd
mdp_partei_TB = df_kommentar[df_kommentar['Fraktion'] == 'AfD']
#mdp_partei = mdp_partei.loc[(mdp_partei['predict'] == 'OFFENSE') & (mdp_partei['date'] >= datetime.date(2019,10,25))]
partei_piv_TB = pd.pivot_table(mdp_partei_TB, columns = ['category'], index = ['Partei'], aggfunc = np.count_nonzero)
partei_piv_TB['Text']
df_kommentar[
#     (df_kommentar['Fraktion'] == 'AfD')
#             &
    (df_kommentar['Woche'] == '20-w12')
            &
#     (df_kommentar['Partei'] != 'AfD')
#             &
    (df_kommentar['category'] == 'neutral')
            ][['Partei', 'Text', 'Rede_ID', 'Fraktion', 'Beschreibung1']]

In [ ]:
import datetime
dp_partei = df_kommentar
mdp_partei_nom = mdp_partei_TB.loc[(mdp_partei_TB['category'] == 'positive') & (mdp_partei_TB['Datum'] >= datetime.date(2019,10,25))]
partei_piv_TB_nom = pd.pivot_table(mdp_partei_nom, columns = ['Fraktion'], index = ['Woche'], aggfunc = np.count_nonzero)
partei_piv_TB_nom = partei_piv_TB_nom['Datum']

mdp_partei_denom = mdp_partei_TB.loc[(mdp_partei_TB['Datum'] >= datetime.date(2019,10,25))]
partei_piv_TB_denom = pd.pivot_table(mdp_partei_denom, columns = ['Fraktion'], index = ['Woche'], aggfunc = np.count_nonzero)
partei_piv_TB_denom = partei_piv_TB_denom['Datum']

partei_piv_TB_rel = (partei_piv_TB_nom/partei_piv_TB_denom) * 100
partei_piv_TB_rel = partei_piv_TB_rel.round()
# partei_piv_TB_rel

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np

category = ['negative', 'neutral', 'positive']
mdp_partei = df_kommentar

plt.style.use('seaborn-darkgrid')
my_palette = ['blue', 'orange', 'green', 'purple', 'red', 'black']

fig = plt.figure(figsize = (20, 15))

mdp_partei_denom = mdp_partei.loc[(mdp_partei['Datum'] >= datetime.date(2019,10,25))]
partei_piv_TB_denom = pd.pivot_table(mdp_partei_denom, columns = ['Partei'], index = ['Woche'], aggfunc = np.count_nonzero)
partei_piv_TB_denom = partei_piv_TB_denom['Datum']

for cat, n in zip(category, range(1,len(category)+1)):
    mdp_partei_nom = mdp_partei.loc[(mdp_partei['category'] == cat) & (mdp_partei['Datum'] >= datetime.date(2019,10,25))]
    partei_piv_TB_nom = pd.pivot_table(mdp_partei_nom, columns = ['Partei'], index = ['Woche'], aggfunc = np.count_nonzero)
    partei_piv_TB_nom = partei_piv_TB_nom['Datum']
    
    partei_piv_TB_rel = (partei_piv_TB_nom/partei_piv_TB_denom) * 100
    partei_piv_TB_rel = partei_piv_TB_rel.round()


###############  plot #################
    ax = fig.add_subplot(3,1,n)

    num=0
    for column in partei_piv_TB_rel:
        ax.plot(partei_piv_TB_rel.index, partei_piv_TB_rel[column], color = my_palette[num], alpha=0.8, label = column)
        num+=1

    ax.plot(partei_piv_TB_rel.index, partei_piv_TB_rel['AfD'], marker='', color='blue', linewidth=4, alpha=0.7)

    ax.set_title('Übersicht der TB relativen Häufigkeit von "' + cat + '-Zurufen"', size = 20)
    ylab = ax.set_ylabel(cat +'/ alle Tweets ', size = 20)
    if n <= 2:
        plt.setp(ax.get_xticklabels(), visible=False)
    else:
        x_tags = plt.xticks(partei_piv_TB_rel.index, fontsize = 14, rotation = 45, ha = "right")
        plt.legend(loc = 3, ncol = 6, fontsize = 17)




In [1]:
# pd.set_option('display.max_colwidth', None)

df_kommentar.loc[(df_kommentar['category'] == 'neutral')
             & (df_kommentar['Partei'] == 'AfD')
             & (df_kommentar['Woche'] == '20-w13')
                ]

NameError: name 'df_kommentar' is not defined